In [ ]:
!pip install requests
import requests
import time
import pandas as pd

In [ ]:
import requests
API_KEY = 'RGAPI-cadd02b1-e5aa-4d07-9e6f-******'  # Replace with your newly regenerated API key
PUUID = 'RijX8s0aF2HTxHHwh1chryqLHyhDQSi-NjOwQlaAhxXE2V8zzrd_ZPdNNV_WcyHTUg7zoZgD5eD5Bw'
MATCH_REGION = 'europe'

headers = {'X-Riot-Token': API_KEY}

#  Get timestamp for 45 days ago
start_time = int(time.time()) - (45 * 24 * 60 * 60)

#  Pagination setup - riot api only supports multiple requests at once so we use pagination
all_match_ids = []
batch_size = 100
start = 0
max_batches = 10  # Limit to 1000 matches max for safety

while True:
    url = f"https://{MATCH_REGION}.api.riotgames.com/lol/match/v5/matches/by-puuid/{PUUID}/ids"
    params = {
        "start": start,
        "count": batch_size,
        "startTime": start_time
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        batch_ids = response.json()
        if not batch_ids:
            break  # No more matches
        all_match_ids.extend(batch_ids)
        print(f"🧩 Fetched {len(batch_ids)} matches at start={start}")
        start += batch_size
        time.sleep(1.5)  # Sleep to avoid hitting rate limit
    elif response.status_code == 429:
        print("⏳ Rate limit hit. Waiting 10 seconds...")
        time.sleep(10)
    else:
        print(f"❌ Error {response.status_code}: {response.text}")
        break

print(f"\n✅ Total matches fetched from last 45 days: {len(all_match_ids)}")

# 📊 Step 2: Fetch match details and collect stats
match_data_list = []

for match_id in all_match_ids:
    match_url = f"https://{MATCH_REGION}.api.riotgames.com/lol/match/v5/matches/{match_id}"
    response = requests.get(match_url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        info = data['info']
        for p in info['participants']:
            if p['puuid'] == PUUID:
                match_data_list.append({
                    'Match ID': match_id,
                    'Game Mode': info.get('gameMode', 'Unknown'),
                    'Duration (min)': round(info.get('gameDuration', 0) / 60, 1),
                    'Champion': p['championName'],
                    'Kills': p['kills'],
                    'Deaths': p['deaths'],
                    'Assists': p['assists'],
                    'Win': p['win'],
                    'Queue ID': info.get('queueId', 'N/A'),
                    'Timestamp': info.get('gameStartTimestamp')
                })
                break
        time.sleep(1.25)  # Small delay to stay within limits
    elif response.status_code == 429:
        print("⏳ Match detail rate limit hit, sleeping 10 sec...")
        time.sleep(10)
    else:
        print(f"❌ Error fetching match {match_id}: {response.status_code}")

# 📁 Step 3: Save to CSV
df = pd.DataFrame(match_data_list)
df.to_csv("riot_match_history_45_days.csv", index=False)
print("\n✅ Match history saved to riot_match_history_45_days.csv")

# Optionally preview first rows
df.head()

🧩 Fetched 100 matches at start=0
🧩 Fetched 100 matches at start=100
🧩 Fetched 1 matches at start=200

✅ Total matches fetched from last 45 days: 201

✅ Match history saved to riot_match_history_45_days.csv


,Match ID,Game Mode,Duration (min),Champion,Kills,Deaths,Assists,Win,Queue ID,Timestamp
0,TR1_1613311076,CLASSIC,25.0,Ambessa,10,5,0,False,440,1746562794201
1,TR1_1613294979,CLASSIC,32.7,Kayle,4,8,4,True,440,1746559928213
2,TR1_1613292360,CLASSIC,16.9,Nasus,0,4,3,True,440,1746558587812
3,TR1_1612732499,CLASSIC,29.2,DrMundo,5,7,12,True,420,1746316242328
4,TR1_1612722439,CLASSIC,41.0,TahmKench,1,10,20,True,420,1746313261032
